In [95]:
# import pandas as pd
# import numpy as np
# from datetime import timedelta

# Task

- If participant A has segmented data, check if segments really belong to participant A

## How?

- check for intersections of timestamps

In [96]:
# name1.split("_")[0] == name2.split("_")[0]

In [97]:
# print(name1.split("_")[0])
# print(name2.split("_")[0])

In [98]:
# root_dir = "data_csv/live/"

# df_1 = pd.read_csv(f"{root_dir}8-PMPerson8D1-live_EPOCX_223800_2024.06.05T15.19.02+08.00.md.mc.pm.fe.bp.csv", header = 1)
# df_1["Timestamp"] = pd.to_datetime(df_1["Timestamp"], unit='s') + timedelta(hours=8)
# df_2 = pd.read_csv(f"{root_dir}8-PMPerson8D1-live_EPOCX_223800_2024.06.05T15.19.33+08.00.md.mc.pm.fe.bp.csv", header = 1)
# df_2["Timestamp"] = pd.to_datetime(df_2["Timestamp"], unit='s') + timedelta(hours=8)
# df_3 = pd.read_csv(f"{root_dir}8-PMPerson8D1-live_EPOCX_223800_2024.06.05T15.20.50+08.00.md.mc.pm.fe.bp.csv", header = 1)
# df_3["Timestamp"] = pd.to_datetime(df_3["Timestamp"], unit='s') + timedelta(hours=8)

In [99]:
# df_1_timestamps = set(df_1.Timestamp) #epoch representation
# df_2_timestamps = set(df_2.Timestamp)
# df_3_timestamps = set(df_3.Timestamp) #epoch representation

In [100]:
# len(df_2_timestamps & df_1_timestamps & df_3_timestamps) == 0

In [101]:
# df_1["Timestamp"]

In [102]:
# df_2["Timestamp"] - df_1["Timestamp"]

In [103]:
# csv_path1,csv_path2
# csv_path3, csv_path4


## Clyde's Code

In [104]:
import pandas as pd
import numpy as np
from datetime import timedelta

In [105]:
# Define the root directory and list of filenames
root_dir = "data_csv/live/"
filenames = [
    "8-PMPerson8D1-live_EPOCX_223800_2024.06.05T15.19.02+08.00.md.mc.pm.fe.bp.csv",
    "8-PMPerson8D1-live_EPOCX_223800_2024.06.05T15.19.33+08.00.md.mc.pm.fe.bp.csv",
    "8-PMPerson8D1-live_EPOCX_223800_2024.06.05T15.20.50+08.00.md.mc.pm.fe.bp.csv"
]

In [106]:
# Initialize a list to store dataframes and timestamps
dataframes = []
all_timestamps = pd.Series(dtype='datetime64[ns]')
timestamp_sources = []

# Loop through the filenames, read each CSV, and process the Timestamp
for idx, filename in enumerate(filenames):
    df = pd.read_csv(f"{root_dir}{filename}", header=1)
    df["Timestamp"] = pd.to_datetime(df["Timestamp"], unit='s') + timedelta(hours=8)
    dataframes.append(df)
    all_timestamps = pd.concat([all_timestamps, df["Timestamp"]])
    timestamp_sources.extend([(timestamp, idx) for timestamp in df["Timestamp"]])

# Convert to DataFrame to check for duplicates with source info
timestamp_df = pd.DataFrame(timestamp_sources, columns=["Timestamp", "Source"])

In [108]:
# Check for duplicates in the combined series
duplicates = timestamp_df.duplicated(subset="Timestamp", keep=False)

# If there are any True values in 'duplicates', it means there are overlapping timestamps
if duplicates.any():
    print("There are overlapping timestamps.")
    overlapping_timestamps = timestamp_df[duplicates]

    # Group by Timestamp to find out which files contain the overlaps
    grouped = overlapping_timestamps.groupby("Timestamp")["Source"].apply(list)

    for timestamp, sources in grouped.items():
        overlapping_files = [filenames[source] for source in sources]
        print(f"Timestamp {timestamp} is overlapping in files: {', '.join(overlapping_files)}")
else:
    print("There are no overlapping timestamps.")


There are no overlapping timestamps.
